In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [4]:
from keras.datasets import reuters

Using TensorFlow backend.


In [13]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [14]:
print('Train:')
print(train_data.shape)
print(train_labels.shape)
print('Test')
print(test_data.shape)
print(test_labels.shape)

Train:
(8982,)
(8982,)
Test
(2246,)
(2246,)


In [33]:
print(np.max(np.max(train_data)))
print(np.max(np.max(test_data)))

9975
9931


In [15]:
print(train_data[0])

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [16]:
print(len(train_data[0]))
print(len(train_data[1]))

87
56


In [40]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [42]:
def print_decoded_newswire(index):
    decoded_newswire = " ".join([reverse_word_index.get(i - 3, '?') for i in train_data[index]])
    print(decoded_newswire)

In [43]:
print_decoded_newswire(22)

? ? inc said it sold to an investor group led by mark hammer 200 000 shares of company common stock and warrants to purchase an additional 200 000 shares ? said the transaction increases the group's stake from 12 9 pct to 18 5 pct the net proceeds to the company total 2 8 mln dlrs the company said the warrants are ? over the next three years at 19 50 dlrs per share of common stock according to ? reuter 3
